In [9]:
from transformers import TimeSeriesTransformerConfig, TimeSeriesTransformerModel, TimeSeriesTransformerForPrediction

from datasets import load_dataset

from datetime import date, datetime


In [10]:
dataset = load_dataset('dataset/index', data_files='GSPC.csv')

filtered_dataset = dataset['train'].filter(lambda e: date.fromisoformat(e['Date'][:10]) > date(2000, 1, 1))
filtered_dataset = filtered_dataset.filter(lambda e: date.fromisoformat(e['Date'][:10]) < date(2023, 1, 1))
train_dataset = filtered_dataset

train_dataset


Using custom data configuration index-c202ce2a383faf27
Found cached dataset csv (/Users/lucyxu/.cache/huggingface/datasets/csv/index-c202ce2a383faf27/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at /Users/lucyxu/.cache/huggingface/datasets/csv/index-c202ce2a383faf27/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-0f0ff6cae82dea89.arrow
Loading cached processed dataset at /Users/lucyxu/.cache/huggingface/datasets/csv/index-c202ce2a383faf27/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-43e00ff25d3bc6a1.arrow


Dataset({
    features: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits'],
    num_rows: 5787
})

In [ ]:
from transformers import PretrainedConfig

# def create_transformation(freq: str, config: PretrainedConfig) -> Transformation:
    # pass


In [ ]:
# transformation = create_transformation(freq, config)
# transformed_data = transformation.apply(train_dataset, is_train=True)



In [30]:
# Initializing a default Time Series Transformer configuration
configuration = TimeSeriesTransformerConfig(
    prediction_length=1,
    context_length=9,

    num_time_features=2,
    num_static_categorical_features=0,
    num_static_real_features=0,
)

# Randomly initializing a model (with random weights) from the configuration
model = TimeSeriesTransformerModel(configuration)

# Accessing the model configuration
configuration = model.config

model


TimeSeriesTransformerModel(
  (scaler): MeanScaler()
  (embedder): FeatureEmbedder(
    (embedders): ModuleList(
      (0): Embedding(1, 1)
    )
  )
  (encoder): TimeSeriesTransformerEncoder(
    (layers): ModuleList(
      (0): TimeSeriesTransformerEncoderLayer(
        (self_attn): TimeSeriesTransformerAttention(
          (k_proj): Linear(in_features=12, out_features=12, bias=True)
          (v_proj): Linear(in_features=12, out_features=12, bias=True)
          (q_proj): Linear(in_features=12, out_features=12, bias=True)
          (out_proj): Linear(in_features=12, out_features=12, bias=True)
        )
        (self_attn_layer_norm): LayerNorm((12,), eps=1e-05, elementwise_affine=True)
        (activation_fn): GELUActivation()
        (fc1): Linear(in_features=12, out_features=32, bias=True)
        (fc2): Linear(in_features=32, out_features=12, bias=True)
        (final_layer_norm): LayerNorm((12,), eps=1e-05, elementwise_affine=True)
      )
      (1): TimeSeriesTransformerEncode

In [33]:
model.config.prediction_length



1

In [31]:
import torch


x = torch.rand((2, 9))
y = torch.rand((2, 1))
mask = torch.ones((2, 9))

time_features = torch.rand((2, 9, 1))

e = torch.empty((2, 0))

outputs = model(
    past_values=x,
    past_time_features=time_features,
    past_observed_mask=mask,
    # future_values=y,
    static_categorical_features=e,
    static_real_features=e,
    output_hidden_states=True,
)


AssertionError: (('input length 9 and dynamic feature lengths 16 does not match',), 9, 16)